In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from weight_discovery import *
from free_diffusion import *

In [ ]:
init_params = [.2,.3,.025]
#init_params = [1,.1,.1]
#ini_params = [1,.6,.1]
plt.plot(realepr(np.linspace(0,.25,1000),*init_params))
params['init'] = init_params

In [ ]:
#Parameters
#sim params
T = .25# Total time
dt = 0.001
num_steps = int(T / dt)
num_paths = 500_000

sim_params = [num_paths, num_steps, dt]

params['sim'] = sim_params


In [ ]:
# Simulate the process
#this will skip steps when saving the output to save memory
skip = 1

process = simulate_free_diffusion_underdamped(params, save_skip=skip)
process.shape


In [ ]:
num_est = 6
time_step = 20

proc_dt = dt * skip

ep = get_step_epr(process, time_step, proc_dt, params['gamma'], num_est)

In [ ]:
fig, ax = plt.subplots()
theo = realepr(np.arange(0,T,proc_dt),*params['init'] )

ax.plot(np.arange(0,T,proc_dt),theo, c='k')
ax.scatter(time_step*proc_dt, ep, marker='D')

In [ ]:

num_est = 10

#steps = [10, 50, 100, 200, 300, 400, 485 ]
steps = [ int(5*i) for i in  [1, 2, 4.2, 10, 15, 20, 25, 35, 47]]
eps=[]
for time_step in steps:
    eps.append(get_step_epr(process, time_step, proc_dt, params['gamma'], num_est))


In [ ]:
fig, ax = plt.subplots()
ax.plot(np.arange(0,T,proc_dt),theo, c='k')


ax.scatter([s*proc_dt for s in steps], eps, marker='D')

In [ ]:
num_iter = 100

num_ests = [4, 6, 9, 10]
steps = [1, 3, 5, 15, 30, 100]
all_estimates = []

for i in range(num_iter):
    process = simulate_free_diffusion_underdamped(num_paths, num_steps, dt, *ini_params, save_skip=skip)
    print(f'sim {i} done', end='\r')

    estimates = []
    for num_est in num_ests:
        n_resolved_estimates=[]
        for time_step in steps:
            ep = get_step_epr(process, time_step, proc_dt, gamma, num_est)
            n_resolved_estimates.append(ep)
        estimates.append(n_resolved_estimates)
        print(f'finished {i} run of {num_est} estimates', end='\r')
    
    all_estimates.append(estimates)


In [ ]:
all_e = np.array(all_estimates)
means = all_e.mean(axis=0)
stds = all_e.std(axis=0)


In [ ]:
all_e.shape


In [ ]:
'''
keys = ['estimates', 'N', 'p0,Sp0,Sx0', 'dt', 'proc_dt', 'num_est', 'steps', 'gamma']
vals = [all_e, num_paths, ini_params, dt, proc_dt, num_ests, steps, gamma]

save_dict = {k:v for k,v in zip(keys, vals)}
kt.save_as_json(save_dict, 'free_diff_monotonic')
'''

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,10))

theo = realepr(np.arange(0,T,proc_dt),*ini_params )



for i in range(0,4):
    ax[0].plot(np.arange(0,T,proc_dt),theo, c='k', linestyle='--')
    ax[0].errorbar([s*proc_dt for s in steps], means[i,:], yerr=3*stds[i,:]/np.sqrt(num_iter), marker='D', label=f'{num_ests[i]} estimates', linestyle='none' )

    ax[1].errorbar([proc_dt*(s+i/6) for s in steps], (means[i,:]-theo[steps])/theo[steps], yerr=3*stds[i,:]/np.sqrt(num_iter)/theo[steps], marker='D', linestyle='none' )
    #ax[1].errorbar([proc_dt*(s+i/6) for s in steps], (means[i,:]-theo[steps]), yerr=3*stds[i,:]/np.sqrt(num_iter), marker='D', linestyle='none' )



fig.legend()

#for i, e in zip(idxs,estimates):
#    ax.scatter(i, e , color='r', marker='D')

ax[0].set_yscale('log')
ax[1].set_xscale('log')
fig.suptitle(f'N={num_paths}, dt={proc_dt}, params={ini_params}, n_iteration={num_iter}')

ax[1].axhline(0, c='k')
ax[0].set_xlabel('t')
ax[0].set_ylabel('entropy rate')
ax[1].set_xlabel('time step')
ax[1].set_ylabel('percent error')

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,10))

theo = realepr(np.arange(0,T,proc_dt),*ini_params )



for i in range(0,4):
    ax[0].plot(np.arange(0,T,proc_dt),theo, c='k', linestyle='--')
    ax[0].errorbar([s*proc_dt for s in steps], means[i,:], yerr=3*stds[i,:]/np.sqrt(num_iter), marker='D', label=f'{num_ests[i]} estimators', linestyle='none' )

    ax[1].errorbar([proc_dt*(s+i/6) for s in steps], (means[i,:]-theo[steps])/theo[steps], yerr=3*stds[i,:]/np.sqrt(num_iter)/theo[steps], marker='D', linestyle='none' )


fig.legend()

#for i, e in zip(idxs,estimates):
#    ax.scatter(i, e , color='r', marker='D')

#ax[0].set_yscale('log')
ax[1].set_xscale('log')
fig.suptitle(f'N={num_paths}, dt={proc_dt}, params={ini_params}, n_iteration={num_iter}')

ax[1].axhline(0, c='k')
ax[0].set_xlabel('t')
ax[0].set_ylabel('entropy rate')
ax[1].set_xlabel('time step')
ax[1].set_ylabel('percent error')



In [ ]:
#fig.savefig('ep_estimation_mono_.pdf')